# Worker/Helmet/Vest Detection using Faster RCNN

In [ ]:
# !pip install pyyaml==5.1
# !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
# assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

In [ ]:
# Assumption: matplotlib, numpy, opencv are installed

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import glob
import matplotlib.pyplot as plt
import numpy as np
import os, json, cv2, random
from PIL import Image
import tqdm

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
import random
random.seed(1364)

# Train on a custom dataset

### Download the dataset

https://github.com/ciber-lab/pictor-ppe

#### Direct link to the dataset:

https://drive.google.com/drive/folders/1M8nzvcnAsEXwz81x18X_mGeqPztZBIvO?usp=sharing

#### Unzip the downloaded files

In [ ]:
# !unzip Images-20210810T232206Z-001.zip -d worker_helmet_vest_dataset
# !unzip Labels-20210810T234322Z-001.zip -d worker_helmet_vest_dataset/Labels

#### An example image

In [ ]:
img = Image.open('worker_helmet_vest_dataset/Images/image_from_china(4182).jpg')
img = np.array(img)
plt.imshow(img)
plt.show()

### Read the annotations for each split

In [ ]:
with open('worker_helmet_vest_dataset/Labels/pictor_ppe_crowdsourced_approach-02_train.txt', 'r') as f:
    train_set = f.readlines()
with open('worker_helmet_vest_dataset/Labels/pictor_ppe_crowdsourced_approach-02_valid.txt', 'r') as f:
    val_set = f.readlines()
with open('worker_helmet_vest_dataset/Labels/pictor_ppe_crowdsourced_approach-02_test.txt', 'r') as f:
    test_set = f.readlines()

In [ ]:
import re
train_set = [re.sub('\t', ' ', line.strip()) for line in train_set]
val_set = [re.sub('\t', ' ', line.strip()) for line in val_set]
test_set = [re.sub('\t', ' ', line.strip()) for line in test_set]

In [ ]:
dataset_split = {}
dataset_split['train'] = {
    re.findall(r'(image_from_china\(\d+\).jpg).*', line)[0]: 
        [
            [int(item) for item in instance.split(',')] 
                for instance in re.findall(r'image_from_china\(\d+\).jpg (.*)', line)[0].split()
        ] 
    for line in train_set
}
#
dataset_split['val'] = {
    re.findall(r'(image_from_china\(\d+\).jpg).*', line)[0]: 
        [
            [int(item) for item in instance.split(',')] 
                for instance in re.findall(r'image_from_china\(\d+\).jpg (.*)', line)[0].split()
        ] 
    for line in val_set
}
#
dataset_split['test'] = {
    re.findall(r'(image_from_china\(\d+\).jpg).*', line)[0]: 
        [
            [int(item) for item in instance.split(',')] 
                for instance in re.findall(r'image_from_china\(\d+\).jpg (.*)', line)[0].split()
        ] 
    for line in test_set
}

### Register train/val/test datasets (converting arbitrary dataset formats to COCO format)

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")

from detectron2.structures import BoxMode

def get_worker_dataset_dicts(split):
    dataset_dicts = []
    
    for filename, image_instances in tqdm.tqdm(dataset_split[split].items(), total=len(dataset_split[split]), desc=f'{split}_data loading'):
        record = {}
        
        full_filename = os.path.join('worker_helmet_vest_dataset', 'Images', filename)
        img = Image.open(full_filename)
        img = np.array(img)
        height, width = img.shape[:2]
        
        record["file_name"] = full_filename
        record["image_id"] = 0
        record["height"] = height
        record["width"] = width
      
        objs = []
        for instance in image_instances:
            px = [instance[0], instance[2]]
            py = [instance[1], instance[3]]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
#                 "segmentation": [poly],
                "category_id": instance[-1], 
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

DatasetCatalog.clear()

for split in ["train", "val", "test"]:
    DatasetCatalog.register("worker_dataset_" + split, lambda split=split: get_worker_dataset_dicts(split))
    MetadataCatalog.get("worker_dataset_" + split).set(thing_classes=["worker", "worker_helmet", "worker_vest", "worker_helmet_vest"])
worker_dataset_train_metadata = MetadataCatalog.get("worker_dataset_train")
worker_dataset_val_metadata = MetadataCatalog.get("worker_dataset_val")
worker_dataset_test_metadata = MetadataCatalog.get("worker_dataset_test")

In [ ]:
dataset_dicts = get_worker_dataset_dicts('train')
worker_dataset_metadata = worker_dataset_train_metadata

In [ ]:
for d in random.sample(dataset_dicts, 3):
    img = Image.open(d['file_name'])
    img = np.array(img)
    visualizer = Visualizer(img, metadata=worker_dataset_metadata, scale=1)
    out = visualizer.draw_dataset_dict(d)
    print(out.get_image().shape)
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    imgPIL = Image.fromarray(out.get_image())
    imgPIL.save(f"sample_gt_{os.path.splitext(os.path.basename(d['file_name']))[0]}.jpg")
    ax.imshow(out.get_image())
    plt.show()

## Training

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader, build_detection_train_loader

class CocoTrainer(DefaultTrainer):

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("eval_dir", exist_ok=True)
            output_folder = "eval_dir"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

#### Instantiate a Faster-RCNN config object

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("worker_dataset_train",)
cfg.DATASETS.VAL = ("worker_dataset_val",)
cfg.DATASETS.TEST = ("worker_dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 5001    
cfg.TEST.EVAL_PERIOD = 500
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

#### Actual training

In [ ]:
%%time
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

## Inference/Evaluation

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
# import the COCO Evaluator to use the COCO Metrics
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Call the COCO Evaluator function and pass the Validation Dataset
evaluator = COCOEvaluator("worker_dataset_val", cfg, False, output_dir="output/")
val_loader = build_detection_test_loader(cfg, "worker_dataset_val")

# Use the created predicted model in the previous step
inference_on_dataset(predictor.model, val_loader, evaluator)

#### Sample Visualisation

In [ ]:
dataset_dicts = get_worker_dataset_dicts('test')
worker_dataset_metadata = worker_dataset_test_metadata

In [ ]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(dataset_dicts, 3):
    img = Image.open(d["file_name"])
    img = np.array(img)
    outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    print(outputs["instances"])
    v = Visualizer(img,
                   metadata=worker_dataset_metadata, 
                   scale=1, 
                   instance_mode=ColorMode.IMAGE 
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    imgPIL = Image.fromarray(out.get_image())
    imgPIL.save(f"sample_pred_fasterrcnn_{os.path.splitext(os.path.basename(d['file_name']))[0]}.jpg")
    plt.imshow(out.get_image())
    plt.show()
    # print(outputs["instances"].to("cpu"))